In [1]:
import pandas as pd
import numpy as np

In [2]:
data     = pd.read_csv('flights.csv')
airports = pd.read_csv("airports.csv")
airlines = pd.read_csv("airlines.csv")

C:\Apps\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
delay_data = data[data['CANCELLED'] == 0].copy()
delay_data = delay_data.drop(columns = ['CANCELLED','CANCELLATION_REASON'])
delay_data = delay_data[delay_data['DIVERTED'] == 0].copy()
delay_data.iloc[:,-5:] = delay_data.iloc[:,-5:].copy().fillna(0)
delay_data.loc[delay_data['ARRIVAL_DELAY'] >= 15,'DELAYED'] = 1
delay_data.loc[delay_data['ARRIVAL_DELAY'] < 15,'DELAYED'] = 0
delay_data = delay_data.drop(columns = ['YEAR','FLIGHT_NUMBER','TAIL_NUMBER','TAXI_OUT','WHEELS_OFF','WHEELS_ON','TAXI_IN','DIVERTED'])

In [4]:
delay_data['DELAYED'].value_counts()

0.0    4650569
1.0    1063439
Name: DELAYED, dtype: int64

In [5]:
# Creating a dataset with AIRLINES as categorical explanatory, SCHEDULED_DEPARTURE as the continuous Explanatory -
df = delay_data.copy()[['AIRLINE','SCHEDULED_DEPARTURE','ORIGIN_AIRPORT','DESTINATION_AIRPORT','DELAYED']].copy()

In [6]:
# One Hot Encoding Cateogorical Data (Airline)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense

In [7]:
df['ROUTE'] = df['ORIGIN_AIRPORT'].astype(str) + '-' + df['DESTINATION_AIRPORT'].astype(str)

In [8]:
df.columns

Index(['AIRLINE', 'SCHEDULED_DEPARTURE', 'ORIGIN_AIRPORT',
       'DESTINATION_AIRPORT', 'DELAYED', 'ROUTE'],
      dtype='object')

In [9]:
df.drop(['ROUTE'],axis=1)

,AIRLINE,SCHEDULED_DEPARTURE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DELAYED
0,AS,5,ANC,SEA,0.0
1,AA,10,LAX,PBI,0.0
2,US,20,SFO,CLT,0.0
3,AA,20,LAX,MIA,0.0
4,AS,25,SEA,ANC,0.0
...,...,...,...,...,...
5819074,B6,2359,LAX,BOS,0.0
5819075,B6,2359,JFK,PSE,0.0
5819076,B6,2359,JFK,SJU,0.0
5819077,B6,2359,MCO,SJU,0.0


In [10]:
df = df[['AIRLINE','ROUTE','DELAYED']].astype(str).copy()

In [11]:
# Defining X and y datasets
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
# y = np.array(y).reshape(len(y),1)

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [12]:
ohe.fit_transform(np.array(X['ORIGIN_AIRPORT'].astype(str)).reshape(-1,1))

NameError: name 'ohe' is not defined

In [13]:
ohe = OneHotEncoder(handle_unknown = 'ignore')
le = LabelEncoder()

ohe.fit(X_train)
le.fit(y_train)

X_train_enc = ohe.transform(X_train)
X_test_enc = ohe.transform(X_test)
y_train_enc = le.transform(y_train)
y_test_enc = le.transform(y_test)

In [14]:
model = Sequential()

In [15]:
# define the  model
model.add(Dense(10, input_dim=X_train_enc.shape[1], activation='relu', kernel_initializer='he_normal'))
model.add(Dense(8,  activation='relu', kernel_initializer='he_normal'))
model.add(Dense(7,  activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))

In [16]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(X_train_enc, y_train_enc, epochs=1, batch_size=1000, verbose=1)



3829/3829 [==============================] - 57s 11ms/step - loss: 0.4913 - accuracy: 0.8137


In [17]:
# evaluate the keras model
_, accuracy = model.evaluate(X_test_enc, y_test_enc, verbose=1)
print('Accuracy: %.2f' % (accuracy*100))

58926/58926 [==============================] - 89s 2ms/step - loss: 0.4704 - accuracy: 0.8136
Accuracy: 81.36


In [ ]:
df['DELAYED'].sum()/df.shape[0]

In [ ]:
df.shape

(5714008, 3)

In [ ]:
df['DELAYED'].sum()